# Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import ast

# Populārāko filmu aprēķins pag. nedēļā

In [2]:
df = pd.read_csv(r"\Dati_mag_darbam\populārākie_raidījumi_ned_test.csv")
df.rename(columns={"movieid":"movieId","userid":"idd_wh_service"},inplace=True)
df.shape

(394, 4)

In [3]:
df.head()

,movieId,channel_name,genre_en,cnt
0,13 going on 30,VIP Comedy,[Comedy],1166
1,три жизни виктора сухорукова,ТВ Центр international,[Drama],923
2,evolution,Sony Turbo,"[Comedy, Sci-Fi]",20
3,robins huds,Filmzone,"[Drama, Action, Adventure, Thriller]",497
4,the eye,TV1000 Action,"[Drama, Mystery, Thriller, Horror]",97


In [7]:
print(df.shape)
df2=df.sort_values(by=['cnt'],ascending=False).head(15).drop(columns='cnt')

(394, 4)


In [8]:
#top 10 filmas
df2

,movieId,channel_name,genre_en
44,calvin & kaison's play power,Nick Jr,[Family]
202,rambo first blood part ii,TV6,"[Action, Adventure, Thriller]"
337,zosu gane,LTV1,[Family]
309,tomasa krauna afera,STV Pirmā!,[Crime]
183,otīlija fon fābere kastela 2 daļa,LTV1,"[Biography, Drama, Historical]"
391,шальные браки,ТВ Центр international,[Drama]
179,non stop,TV3,"[Action, Mystery, Thriller]"
359,ирина печерникова от первой до последней любви,ТВ Центр international,[Drama]
161,misters tvisters uz skatuves,LTV1,"[Drama, Comedy, Family]"
298,the titan,3+,"[Drama, Romance, Mystery, Thriller, Sci-Fi]"


# Lietotāja preferences

In [9]:
#lietotāju preferenču aprēķins, ņemot lietotāja skatītāko stundu, dienu un žanru kombināciju, kā arī aprēķināti lietotāja lietotie kanāli
pref = pd.read_csv(r"\Dati_mag_darbam\user_value_calc.csv")
list_parameters=['genre_en']
for item in list_parameters:
    pref[item]=pref[item].str.replace(' ','').str.strip('[]').str.split(',')
    pref=pref.explode(item)
pref_kanali=pref.groupby(['idd_wh_service','channel_name'])['sum_len_broadcast_h_calc'].sum().reset_index()
pref=pref.groupby(['hour', 'is_weekend', 'genre_en', 'idd_wh_service'])['sum_len_broadcast_h_calc'].sum().reset_index()
pref.sort_values(by=['idd_wh_service','sum_len_broadcast_h_calc'],ascending=False,inplace=True)
pref=pref.groupby('idd_wh_service').head(10).reset_index(drop=True)

In [10]:
#preferenču piemērs
pref[pref.idd_wh_service=='DTVS0223886']

,hour,is_weekend,genre_en,idd_wh_service,sum_len_broadcast_h_calc
799981,18-23,Nē,Drama,DTVS0223886,10.579722
799982,18-23,Nē,Action,DTVS0223886,9.347500
799983,18-23,Nē,Thriller,DTVS0223886,8.086944
799984,18-23,Nē,Romance,DTVS0223886,6.971389
799985,18-23,Jā,Drama,DTVS0223886,6.416111
799986,12-18,Jā,Adventure,DTVS0223886,5.270833
799987,12-18,Jā,Family,DTVS0223886,5.270833
799988,18-23,Nē,Sci-Fi,DTVS0223886,5.265000
799989,12-18,Jā,Drama,DTVS0223886,4.863611
799990,12-18,Jā,Fantasy,DTVS0223886,4.217500


In [12]:
#populārāko skatīšanās laiku aprēķins iepriekšējā mēneša laikā - top 15 pozīcijas katrā kanālā
pop_param = pd.read_csv(r"\Dati_mag_darbam\populārākā_satura_parametri_menesis.csv")
list_parameters=['genre_en']
for item in list_parameters:
    pop_param[item]=pop_param[item].str.replace(' ','').str.strip('[]').str.split(',')
    pop_param=pop_param.explode(item)
    pop_param.groupby(['hour', 'is_weekend', 'channel_name', 'genre_en'])['cnt'].sum().reset_index(drop=True,inplace=True)
    pop_param.sort_values(by=['channel_name','cnt'],ascending=False,inplace=True)
    pop_param=pop_param.groupby('channel_name').head(15).reset_index(drop=True)

In [14]:
#pieejamā satura parametru apstrāde
pieejams = pd.read_csv(r"\Dati_mag_darbam\next_week_avlbl.csv")
list_parameters=['genre_en']
for item in list_parameters:
    pieejams[item]=pieejams[item].str.replace(' ','').str.strip('[]').str.split(',')
    pieejams=pieejams.explode(item)
pieejams.rename(columns={"movieid":"movieId"},inplace=True)
print(pieejams.shape)
pieejams.head()

(3156, 6)


,hour,is_weekend,channel_name,genre_en,movieId,dt_epg_start
0,23-01,Jā,РТР-Планета,Drama,деревенщина,2021-04-04
1,6-9,Jā,360TV,Action,spiegu berni,2021-04-04
1,6-9,Jā,360TV,Adventure,spiegu berni,2021-04-04
1,6-9,Jā,360TV,Comedy,spiegu berni,2021-04-04
1,6-9,Jā,360TV,Family,spiegu berni,2021-04-04


In [15]:
#aprēķināts populārs un pieejams saturs
pop_un_pieejams=pieejams.merge(pop_param, how='inner', on=['hour','is_weekend','channel_name','genre_en'])
pop_un_pieejams.shape

(1074, 7)

In [16]:
#lietotāja preferences apvienotas ar pieejamo un populāro saturu. Pievienotas arī lietotāja kanālu preferences
pop_ieteikumi=pref.merge(pop_un_pieejams, how='inner', on=['hour','is_weekend','genre_en'])
pop_ieteikumi=pop_ieteikumi.merge(pref_kanali.rename(columns={'sum_len_broadcast_h_calc':'channel_pref'}), how='left', on=['idd_wh_service','channel_name'])

In [17]:
#aprēķināts lietotājam ieteiktais populārais saturs, vispirms kārtojot pēc iecienītākajiem skatīšanās laikiem uc. parametriem
# un lietotāja skatītajiem kanāliem. Ja kanāls nav skatīts, bet pārējie parametri atbilst, šie ieteikumi nāks pēc
#atlasītas 10 ieteiktās vienības
pop_ieteikumi.drop_duplicates(subset=('hour','is_weekend','idd_wh_service','channel_name','movieId'),inplace=True)
pop_ieteikumi.sort_values(by=['idd_wh_service','channel_pref'],ascending=False,inplace=True)
pop_ieteikumi=pop_ieteikumi.groupby('idd_wh_service').head(10).reset_index(drop=True)

In [18]:
pop_ieteikumi.shape

(1425708, 10)

In [19]:
pop_ieteikumi.movieId.nunique()

371

In [20]:
pop_ieteikumi[pop_ieteikumi.idd_wh_service=='DTVS0223886']


,hour,is_weekend,genre_en,idd_wh_service,sum_len_broadcast_h_calc,channel_name,movieId,dt_epg_start,cnt,channel_pref
883824,18-23,Nē,Action,DTVS0223886,9.347500,3+,man of tai chi,2021-04-07,7691,45.086944
883825,18-23,Nē,Action,DTVS0223886,9.347500,3+,lung hing foo dai,2021-04-06,7691,45.086944
883826,12-18,Jā,Adventure,DTVS0223886,5.270833,3+,the darkest minds,2021-04-04,6574,45.086944
883827,12-18,Jā,Adventure,DTVS0223886,5.270833,3+,nim's island,2021-04-10,6574,45.086944
883828,12-18,Jā,Adventure,DTVS0223886,5.270833,3+,jack the giant slayer,2021-04-10,6574,45.086944
883829,12-18,Jā,Adventure,DTVS0223886,5.270833,3+,the cat in the hat,2021-04-04,6574,45.086944
883830,12-18,Jā,Adventure,DTVS0223886,5.270833,3+,storks,2021-04-10,6574,45.086944
883831,12-18,Jā,Adventure,DTVS0223886,5.270833,3+,x men apocalypse,2021-04-03,6574,45.086944
883832,12-18,Jā,Family,DTVS0223886,5.270833,3+,sing,2021-04-03,6574,45.086944
883833,18-23,Nē,Action,DTVS0223886,9.347500,TV3,lucy,2021-04-08,8588,37.328611


In [21]:
movies_seen_test = pd.read_csv(r"\\TELEKOM\LTC_Faili\LTC\KD\BAPD\_Iekšējie dokumenti\09_BigData\Data_driven\Projekti\EPG_Rekomendācijas\1. Dati\Test\movies_seen.csv")

movies_seen_test.rename(columns={"movieid":"movieId","userid":"idd_wh_service"},inplace=True)
movies_seen_test['seen']=1
movies_seen_test['seen_live']=np.where(movies_seen_test['event_name']=='Tiešraide','live','archive')
unit_cnt_seen=movies_seen_test.groupby(['idd_wh_service','seen_live'])['movieId'].count().reset_index().rename(columns={'movieId':'seen_cnt'})
unit_cnt_seen=unit_cnt_seen.pivot_table(values='seen_cnt',index='idd_wh_service',columns='seen_live',aggfunc='sum',fill_value=0).reset_index()
unit_cnt_seen['seen_tot']=unit_cnt_seen['archive']+unit_cnt_seen['live']

In [22]:
evaluate=pop_ieteikumi.merge(movies_seen_test, on=['idd_wh_service','movieId'], how='left')
cnt_seen_from_rek=evaluate.groupby('idd_wh_service')['seen'].count().reset_index().rename(columns={'seen':'seen_from_rec'})
cnt_seen_from_rek.fillna(0,inplace=True)
evaluate_views=unit_cnt_seen.merge(cnt_seen_from_rek, on='idd_wh_service',how='left')

In [23]:
evaluate_views['PROC_SEEN_FROM_REC']=np.where(evaluate_views.seen_tot>10,evaluate_views.seen_from_rec/10,evaluate_views.seen_from_rec/evaluate_views.seen_tot)

In [24]:
print("watched at least one movie in test period")
print(evaluate_views.shape[0])
print("watched at least one recommended")
print(evaluate_views[evaluate_views.seen_from_rec>0].shape[0])
print("seen at least one movie in archive")
print(evaluate_views[evaluate_views['archive']>0].shape[0])
print("seen at least one movie in archive and watched at least one recommended")
print(evaluate_views[(evaluate_views['archive']>0)&evaluate_views.seen_from_rec>0].shape[0])
print("seen 10+ movies")
print(evaluate_views[(evaluate_views.seen_tot>=10)].shape[0])
print("seen 10+ movies and watched at least one recommended")
print(evaluate_views[(evaluate_views.seen_from_rec>0)&(evaluate_views.seen_tot>=10)].shape[0])
print("% from watched that were recommended - if watched at least one")
print(round(evaluate_views[evaluate_views.seen_from_rec>0].PROC_SEEN_FROM_REC.mean()*100))
print("% from watched that were recommended")
print(round(evaluate_views.PROC_SEEN_FROM_REC.mean()*100))
print("seen 10+ movies % from watched that were recommended")
print(round(evaluate_views[(evaluate_views.seen_tot>=10)].PROC_SEEN_FROM_REC.mean()*100))
print("recall @ k")
print(evaluate_views.seen_tot)

watched at least one movie in test period
121185
watched at least one recommended
58454
seen at least one movie in archive
52288
seen at least one movie in archive and watched at least one recommended
30356
seen 10+ movies
18455
seen 10+ movies and watched at least one recommended
15014
% from watched that were recommended - if watched at least one
40
% from watched that were recommended
20
seen 10+ movies % from watched that were recommended
23
recall @ k
0         9
1         8
2         7
3         6
4         1
         ..
121180    2
121181    2
121182    1
121183    3
121184    3
Name: seen_tot, Length: 121185, dtype: int64


In [25]:
evaluate_views.seen_tot

0         9
1         8
2         7
3         6
4         1
         ..
121180    2
121181    2
121182    1
121183    3
121184    3
Name: seen_tot, Length: 121185, dtype: int64

In [26]:
evaluate_views['REC@K']=np.where(evaluate_views.seen_tot>10,evaluate_views.seen_from_rec/10,evaluate_views.seen_from_rec/evaluate_views.seen_tot)

In [27]:
evaluate[evaluate.idd_wh_service=='DTVS2004708']

,hour,is_weekend,genre_en,idd_wh_service,sum_len_broadcast_h_calc,channel_name,movieId,dt_epg_start,cnt,channel_pref,event_name,seen,seen_live
100,18-23,Jā,Drama,DTVS2004708,3.677500,TV3 Life,le week end,2021-04-03,6208,31.050278,NaN,NaN,NaN
101,18-23,Nē,Romance,DTVS2004708,4.836667,TV3 Life,home again,2021-04-05,14451,31.050278,Tiešraide,1.0,live
102,18-23,Nē,Romance,DTVS2004708,4.836667,TV3 Life,home again,2021-04-05,14451,31.050278,Arhīvs,1.0,archive
103,18-23,Nē,Romance,DTVS2004708,4.836667,TV3 Life,the sauce of love,2021-04-06,14451,31.050278,NaN,NaN,NaN
104,18-23,Nē,Romance,DTVS2004708,4.836667,TV3 Life,charie st cloud,2021-04-07,14451,31.050278,NaN,NaN,NaN
105,18-23,Jā,Other,DTVS2004708,5.997500,TV3 Life,return to montauk,2021-04-04,21377,31.050278,NaN,NaN,NaN
106,18-23,Jā,Other,DTVS2004708,5.997500,TV3 Life,die andere tochter,2021-04-10,21377,31.050278,Ieraksts,1.0,archive
107,18-23,Jā,Other,DTVS2004708,5.997500,TV3 Life,my boyfriend's dogs,2021-04-04,21377,31.050278,NaN,NaN,NaN
108,18-23,Jā,Romance,DTVS2004708,4.047500,TV3 Life,love to the rescue,2021-04-03,6208,31.050278,NaN,NaN,NaN
109,12-18,Jā,Comedy,DTVS2004708,5.743611,STV Pirmā!,uzvari zaude mili,2021-04-04,4238,17.043056,NaN,NaN,NaN


# Ieteikumu pārbaude top 10 saturam pagājušajā ned.

In [28]:
users=unit_cnt_seen[['idd_wh_service']]
users['key'] = 0
df2['key'] = 0
df3=users.merge(df2, how="inner", on='key')[['idd_wh_service','movieId']]

C:\Aplikacijas\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
evaluate=df3.merge(movies_seen_test, on=['idd_wh_service','movieId'], how='left')
cnt_seen_from_rek=evaluate.groupby('idd_wh_service')['seen'].count().reset_index().rename(columns={'seen':'seen_from_rec'})
cnt_seen_from_rek.fillna(0,inplace=True)
evaluate_views=unit_cnt_seen.merge(cnt_seen_from_rek, on='idd_wh_service',how='left')

In [30]:
evaluate_views['PROC_SEEN_FROM_REC']=np.where(evaluate_views.seen_tot>10,evaluate_views.seen_from_rec/10,evaluate_views.seen_from_rec/evaluate_views.seen_tot)

In [31]:
print("watched at least one movie in test period")
print(evaluate_views.shape[0])
print("watched at least one recommended")
print(evaluate_views[evaluate_views.seen_from_rec>0].shape[0])
print("seen at least one movie in archive")
print(evaluate_views[evaluate_views['archive']>0].shape[0])
print("seen at least one movie in archive and watched at least one recommended")
print(evaluate_views[(evaluate_views['archive']>0)&evaluate_views.seen_from_rec>0].shape[0])
print("seen 10+ movies")
print(evaluate_views[(evaluate_views.seen_tot>=10)].shape[0])
print("seen 10+ movies and watched at least one recommended")
print(evaluate_views[(evaluate_views.seen_from_rec>0)&(evaluate_views.seen_tot>=10)].shape[0])
print("% from watched that were recommended - if watched at least one")
print(round(evaluate_views[evaluate_views.seen_from_rec>0].PROC_SEEN_FROM_REC.mean()*100))
print("% from watched that were recommended")
print(round(evaluate_views.PROC_SEEN_FROM_REC.mean()*100))
print("seen 10+ movies % from watched that were recommended")
print(round(evaluate_views[(evaluate_views.seen_tot>=10)].PROC_SEEN_FROM_REC.mean()*100))

watched at least one movie in test period
121185
watched at least one recommended
15367
seen at least one movie in archive
52288
seen at least one movie in archive and watched at least one recommended
8022
seen 10+ movies
18455
seen 10+ movies and watched at least one recommended
4970
% from watched that were recommended - if watched at least one
27
% from watched that were recommended
3
seen 10+ movies % from watched that were recommended
3
